In [ ]:
from keras.models import load_model
import cv2
import numpy as np

model = load_model("keras_Model.h5", compile=False)
label_list = open("labels.txt", "r").readlines()

In [ ]:
def data_preprocess(image):
    data = cv2.resize(image, (224, 224), interpolation = cv2.INTER_AREA)
    data = data.reshape(1, 224, 224, 3)
    data = data.astype(np.float32)
    data = (data / 127.5) - 1
    return data

In [ ]:
def predict(model, data):
    predict = model.predict(data)
    index = np.argmax(prediction)
    label = label_list[index][2:]
    predict_probability = predict[0][index]
    return label, predict_probability

In [ ]:
def ser_send(ser, label):
    ser.write(label[0].encode())

In [ ]:
ser = serial.Serial('COM29', 9600)
camera = cv2.VideoCapture(0)

while True:
    ret, image = camera.read()
    image = cv2.flip(image, 1)

    data = data_preprocess(image)
    label, predict_probability = predict(model, data)

    if predict_probability >= 0.7:
        text = f"{label} : {np.round(predict_probability * 100, 2)}%"
        cv2.putText(image, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        ser_send(ser, label)
    
    cv2.imshow("Webcam Image", image)
    
    if cv2.pollKey() == ord('q'):
        break

ser.close()
camera.release()
cv2.destroyAllWindows()